# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Functionta
    end_sequence_id = target_vocab_to_int['<EOS>']
    
    source_id_text = [[source_vocab_to_int[vocab] for vocab in line.split()] for line in 
                      source_text.split('\n')]
    
    target_id_text =  [[target_vocab_to_int[vocab] for vocab in line.split()] + [end_sequence_id] 
                       for line in target_text.split('\n')]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None, None], name='input')
    targets = tf.placeholder(tf.int32, shape=[None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, shape=[None], name = 'target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name = 'max_target_len')
    source_sequence_length = tf.placeholder(tf.int32,  shape=[None], name = 'source_sequence_length')
    return inputs, targets, learning_rate, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'F

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    start_sequence_id = target_vocab_to_int['<GO>']
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    decoding_input = tf.concat([tf.fill([batch_size, 1], start_sequence_id), ending], 1)

    return decoding_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    # Creathe embedding layer.
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    # If we don't have an initial zero state, provide a dtype.
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)
    return output, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    # Create the embedding helper.
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Use the same proess as in the encoding layer.
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 3
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 1
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.55
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.0762, Validation Accuracy: 0.0636, Loss: 4.8076
Epoch   0 Batch    2/1077 - Train Accuracy: 0.1020, Validation Accuracy: 0.0799, Loss: 4.3579
Epoch   0 Batch    3/1077 - Train Accuracy: 0.2520, Validation Accuracy: 0.3210, Loss: 3.9672
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2656, Validation Accuracy: 0.3388, Loss: 3.7920
Epoch   0 Batch    5/1077 - Train Accuracy: 0.2910, Validation Accuracy: 0.3338, Loss: 3.6536
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2789, Validation Accuracy: 0.3391, Loss: 3.5614
Epoch   0 Batch    7/1077 - Train Accuracy: 0.2691, Validation Accuracy: 0.3441, Loss: 3.6106
Epoch   0 Batch    8/1077 - Train Accuracy: 0.2695, Validation Accuracy: 0.3402, Loss: 3.4382
Epoch   0 Batch    9/1077 - Train Accuracy: 0.2770, Validation Accuracy: 0.3409, Loss: 3.3529
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2669, Validation Accuracy: 0.3583, Loss: 3.4469
Epoch   0 Batch   11/1077 - Train Accuracy: 0.3218, Validati

Epoch   0 Batch   89/1077 - Train Accuracy: 0.5184, Validation Accuracy: 0.5334, Loss: 0.8552
Epoch   0 Batch   90/1077 - Train Accuracy: 0.4809, Validation Accuracy: 0.5352, Loss: 0.9008
Epoch   0 Batch   91/1077 - Train Accuracy: 0.5670, Validation Accuracy: 0.5398, Loss: 0.7615
Epoch   0 Batch   92/1077 - Train Accuracy: 0.5149, Validation Accuracy: 0.5412, Loss: 0.8278
Epoch   0 Batch   93/1077 - Train Accuracy: 0.5352, Validation Accuracy: 0.5533, Loss: 0.8489
Epoch   0 Batch   94/1077 - Train Accuracy: 0.5273, Validation Accuracy: 0.5518, Loss: 0.7749
Epoch   0 Batch   95/1077 - Train Accuracy: 0.5387, Validation Accuracy: 0.5550, Loss: 0.8527
Epoch   0 Batch   96/1077 - Train Accuracy: 0.5176, Validation Accuracy: 0.5582, Loss: 0.8095
Epoch   0 Batch   97/1077 - Train Accuracy: 0.5086, Validation Accuracy: 0.5533, Loss: 0.8008
Epoch   0 Batch   98/1077 - Train Accuracy: 0.5751, Validation Accuracy: 0.5604, Loss: 0.7734
Epoch   0 Batch   99/1077 - Train Accuracy: 0.5047, Validati

Epoch   0 Batch  177/1077 - Train Accuracy: 0.6896, Validation Accuracy: 0.6559, Loss: 0.5969
Epoch   0 Batch  178/1077 - Train Accuracy: 0.6809, Validation Accuracy: 0.6609, Loss: 0.5551
Epoch   0 Batch  179/1077 - Train Accuracy: 0.6780, Validation Accuracy: 0.6541, Loss: 0.6086
Epoch   0 Batch  180/1077 - Train Accuracy: 0.6516, Validation Accuracy: 0.6623, Loss: 0.5719
Epoch   0 Batch  181/1077 - Train Accuracy: 0.6328, Validation Accuracy: 0.6545, Loss: 0.5945
Epoch   0 Batch  182/1077 - Train Accuracy: 0.6752, Validation Accuracy: 0.6534, Loss: 0.5670
Epoch   0 Batch  183/1077 - Train Accuracy: 0.6594, Validation Accuracy: 0.6435, Loss: 0.5703
Epoch   0 Batch  184/1077 - Train Accuracy: 0.6773, Validation Accuracy: 0.6463, Loss: 0.5512
Epoch   0 Batch  185/1077 - Train Accuracy: 0.6273, Validation Accuracy: 0.6626, Loss: 0.5664
Epoch   0 Batch  186/1077 - Train Accuracy: 0.6250, Validation Accuracy: 0.6626, Loss: 0.5804
Epoch   0 Batch  187/1077 - Train Accuracy: 0.6426, Validati

Epoch   0 Batch  265/1077 - Train Accuracy: 0.7480, Validation Accuracy: 0.7376, Loss: 0.4129
Epoch   0 Batch  266/1077 - Train Accuracy: 0.7336, Validation Accuracy: 0.7390, Loss: 0.4022
Epoch   0 Batch  267/1077 - Train Accuracy: 0.7667, Validation Accuracy: 0.7376, Loss: 0.3718
Epoch   0 Batch  268/1077 - Train Accuracy: 0.7691, Validation Accuracy: 0.7472, Loss: 0.4159
Epoch   0 Batch  269/1077 - Train Accuracy: 0.7011, Validation Accuracy: 0.7447, Loss: 0.4393
Epoch   0 Batch  270/1077 - Train Accuracy: 0.7004, Validation Accuracy: 0.7326, Loss: 0.4199
Epoch   0 Batch  271/1077 - Train Accuracy: 0.7992, Validation Accuracy: 0.7362, Loss: 0.3809
Epoch   0 Batch  272/1077 - Train Accuracy: 0.7641, Validation Accuracy: 0.7280, Loss: 0.4236
Epoch   0 Batch  273/1077 - Train Accuracy: 0.7563, Validation Accuracy: 0.7301, Loss: 0.3769
Epoch   0 Batch  274/1077 - Train Accuracy: 0.7779, Validation Accuracy: 0.7102, Loss: 0.3835
Epoch   0 Batch  275/1077 - Train Accuracy: 0.7318, Validati

Epoch   0 Batch  353/1077 - Train Accuracy: 0.7722, Validation Accuracy: 0.7731, Loss: 0.3444
Epoch   0 Batch  354/1077 - Train Accuracy: 0.8059, Validation Accuracy: 0.7866, Loss: 0.3339
Epoch   0 Batch  355/1077 - Train Accuracy: 0.7939, Validation Accuracy: 0.7777, Loss: 0.3158
Epoch   0 Batch  356/1077 - Train Accuracy: 0.8133, Validation Accuracy: 0.7749, Loss: 0.3245
Epoch   0 Batch  357/1077 - Train Accuracy: 0.7987, Validation Accuracy: 0.7628, Loss: 0.3051
Epoch   0 Batch  358/1077 - Train Accuracy: 0.7438, Validation Accuracy: 0.7631, Loss: 0.3298
Epoch   0 Batch  359/1077 - Train Accuracy: 0.8004, Validation Accuracy: 0.7596, Loss: 0.3130
Epoch   0 Batch  360/1077 - Train Accuracy: 0.7953, Validation Accuracy: 0.7681, Loss: 0.2919
Epoch   0 Batch  361/1077 - Train Accuracy: 0.8269, Validation Accuracy: 0.7699, Loss: 0.3105
Epoch   0 Batch  362/1077 - Train Accuracy: 0.8017, Validation Accuracy: 0.7795, Loss: 0.2880
Epoch   0 Batch  363/1077 - Train Accuracy: 0.8070, Validati

...

Epoch   2 Batch  667/1077 - Train Accuracy: 0.9511, Validation Accuracy: 0.9332, Loss: 0.0654
Epoch   2 Batch  668/1077 - Train Accuracy: 0.9394, Validation Accuracy: 0.9368, Loss: 0.0597
Epoch   2 Batch  669/1077 - Train Accuracy: 0.9758, Validation Accuracy: 0.9386, Loss: 0.0502
Epoch   2 Batch  670/1077 - Train Accuracy: 0.9638, Validation Accuracy: 0.9379, Loss: 0.0584
Epoch   2 Batch  671/1077 - Train Accuracy: 0.9416, Validation Accuracy: 0.9339, Loss: 0.0658
Epoch   2 Batch  672/1077 - Train Accuracy: 0.9457, Validation Accuracy: 0.9339, Loss: 0.0509
Epoch   2 Batch  673/1077 - Train Accuracy: 0.9345, Validation Accuracy: 0.9357, Loss: 0.0600
Epoch   2 Batch  674/1077 - Train Accuracy: 0.9551, Validation Accuracy: 0.9361, Loss: 0.0473
Epoch   2 Batch  675/1077 - Train Accuracy: 0.9405, Validation Accuracy: 0.9329, Loss: 0.0726
Epoch   2 Batch  676/1077 - Train Accuracy: 0.9618, Validation Accuracy: 0.9297, Loss: 0.0535
Epoch   2 Batch  677/1077 - Train Accuracy: 0.9387, Validati

Epoch   2 Batch  755/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9439, Loss: 0.0697
Epoch   2 Batch  756/1077 - Train Accuracy: 0.9488, Validation Accuracy: 0.9499, Loss: 0.0469
Epoch   2 Batch  757/1077 - Train Accuracy: 0.9424, Validation Accuracy: 0.9432, Loss: 0.0507
Epoch   2 Batch  758/1077 - Train Accuracy: 0.9673, Validation Accuracy: 0.9393, Loss: 0.0467
Epoch   2 Batch  759/1077 - Train Accuracy: 0.9691, Validation Accuracy: 0.9428, Loss: 0.0434
Epoch   2 Batch  760/1077 - Train Accuracy: 0.9504, Validation Accuracy: 0.9471, Loss: 0.0554
Epoch   2 Batch  761/1077 - Train Accuracy: 0.9560, Validation Accuracy: 0.9577, Loss: 0.0532
Epoch   2 Batch  762/1077 - Train Accuracy: 0.9352, Validation Accuracy: 0.9560, Loss: 0.0462
Epoch   2 Batch  763/1077 - Train Accuracy: 0.9606, Validation Accuracy: 0.9503, Loss: 0.0534
Epoch   2 Batch  764/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9446, Loss: 0.0468
Epoch   2 Batch  765/1077 - Train Accuracy: 0.9305, Validati

Epoch   2 Batch  843/1077 - Train Accuracy: 0.9282, Validation Accuracy: 0.9279, Loss: 0.0546
Epoch   2 Batch  844/1077 - Train Accuracy: 0.9587, Validation Accuracy: 0.9212, Loss: 0.0494
Epoch   2 Batch  845/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9286, Loss: 0.0585
Epoch   2 Batch  846/1077 - Train Accuracy: 0.9371, Validation Accuracy: 0.9379, Loss: 0.0770
Epoch   2 Batch  847/1077 - Train Accuracy: 0.9547, Validation Accuracy: 0.9442, Loss: 0.0788
Epoch   2 Batch  848/1077 - Train Accuracy: 0.9578, Validation Accuracy: 0.9506, Loss: 0.0540
Epoch   2 Batch  849/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9478, Loss: 0.0496
Epoch   2 Batch  850/1077 - Train Accuracy: 0.9036, Validation Accuracy: 0.9499, Loss: 0.0894
Epoch   2 Batch  851/1077 - Train Accuracy: 0.9364, Validation Accuracy: 0.9450, Loss: 0.0670
Epoch   2 Batch  852/1077 - Train Accuracy: 0.9109, Validation Accuracy: 0.9421, Loss: 0.0811
Epoch   2 Batch  853/1077 - Train Accuracy: 0.9430, Validati

Epoch   2 Batch  931/1077 - Train Accuracy: 0.9570, Validation Accuracy: 0.9251, Loss: 0.0597
Epoch   2 Batch  932/1077 - Train Accuracy: 0.9516, Validation Accuracy: 0.9254, Loss: 0.0544
Epoch   2 Batch  933/1077 - Train Accuracy: 0.9434, Validation Accuracy: 0.9247, Loss: 0.0608
Epoch   2 Batch  934/1077 - Train Accuracy: 0.9441, Validation Accuracy: 0.9265, Loss: 0.0625
Epoch   2 Batch  935/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9272, Loss: 0.0586
Epoch   2 Batch  936/1077 - Train Accuracy: 0.9394, Validation Accuracy: 0.9290, Loss: 0.0546
Epoch   2 Batch  937/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9300, Loss: 0.0721
Epoch   2 Batch  938/1077 - Train Accuracy: 0.9586, Validation Accuracy: 0.9297, Loss: 0.0626
Epoch   2 Batch  939/1077 - Train Accuracy: 0.9250, Validation Accuracy: 0.9357, Loss: 0.0699
Epoch   2 Batch  940/1077 - Train Accuracy: 0.9395, Validation Accuracy: 0.9382, Loss: 0.0534
Epoch   2 Batch  941/1077 - Train Accuracy: 0.9550, Validati

Epoch   2 Batch 1019/1077 - Train Accuracy: 0.9404, Validation Accuracy: 0.9155, Loss: 0.0747
Epoch   2 Batch 1020/1077 - Train Accuracy: 0.9707, Validation Accuracy: 0.9205, Loss: 0.0366
Epoch   2 Batch 1021/1077 - Train Accuracy: 0.9427, Validation Accuracy: 0.9116, Loss: 0.0543
Epoch   2 Batch 1022/1077 - Train Accuracy: 0.9479, Validation Accuracy: 0.9116, Loss: 0.0454
Epoch   2 Batch 1023/1077 - Train Accuracy: 0.9641, Validation Accuracy: 0.9052, Loss: 0.0437
Epoch   2 Batch 1024/1077 - Train Accuracy: 0.9207, Validation Accuracy: 0.9052, Loss: 0.0768
Epoch   2 Batch 1025/1077 - Train Accuracy: 0.9368, Validation Accuracy: 0.9180, Loss: 0.0503
Epoch   2 Batch 1026/1077 - Train Accuracy: 0.9591, Validation Accuracy: 0.9258, Loss: 0.0672
Epoch   2 Batch 1027/1077 - Train Accuracy: 0.9469, Validation Accuracy: 0.9268, Loss: 0.0474
Epoch   2 Batch 1028/1077 - Train Accuracy: 0.9312, Validation Accuracy: 0.9315, Loss: 0.0510
Epoch   2 Batch 1029/1077 - Train Accuracy: 0.9496, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [24]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    word_ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [25]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [162, 225, 156, 191, 84, 104, 17]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [83, 231, 19, 71, 40, 322, 61, 284, 1]
  French Words: il a vu un vieux camion rouge . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.